In [ ]:
# !pip install selenium
# !pip install chromedriver-autoinstaller
# TODO 첫 화면에서 로그인 후 검색으로 이동하기
# TODO target_region을 list[str]타입으로 하여 한 지역식 검색하도록 하기
# TODO 마지막 페이지를 모두 검색한 후 target_region이 남아있으면 처음으로 돌아가기

In [ ]:
import chromedriver_autoinstaller # chrome driver 자동 설치 라이브러리
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import pandas as pd
import os
from time import sleep

In [ ]:
# chrome driver를 자동으로 설치함
chromedriver_autoinstaller.install()

options = webdriver.ChromeOptions() # Browser 세팅하기
options.add_argument('lang=ko_KR') # 사용언어 한국어
options.add_argument('disable-gpu') # 하드웨어 가속 안함
# options.add_argument('headless') # 창 숨기기

# 브라우저 세팅
driver = webdriver.Chrome(options=options)

In [ ]:
# 브라우저에 URL 호출하기
target_url='https://new.cretop.com/ET/SS/ETSS071M1?h=1683949489638'
driver.get(url=target_url)

In [ ]:
def select_search_option(target_option, target_region):
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'filter-wrap'))
    )
    webEl_filter_wrap = driver.find_element(By.CLASS_NAME, 'filter-wrap')
    # TODO 검색어 상품명 접근하도록 하기

    webEl_filter_title =  webEl_filter_wrap.find_elements(By.CLASS_NAME, 'filter-tit')
    webEl_filter_option =  webEl_filter_wrap.find_elements(By.CLASS_NAME, 'filter-option')

    for opt in webEl_filter_option:
        try:
            for check in opt.find_elements(By.CLASS_NAME, 'form-check'):
                if check.text in target_option:
                    check.click()
                    print(check.text)
        except Exception: 
            print(Exception)
            pass
        for radio in opt.find_elements(By.CLASS_NAME, 'form-radio'): 
            if radio.text == target_region:
                radio.find_element(By.TAG_NAME, "label").click()
                print(radio.text)


In [ ]:
def click_search():
    try:
        close_btn_raw_xpath = '//*[@id="app"]/div[1]/div[1]/div/div/div[1]/div/div/div/div/div[2]/button[1]'
        search_btn_raw_xpath ='//*[@id="app"]/div[1]/div[1]/div/div/div[1]/div/div/div/div/div[2]/button[2]'

    except Exception:
        search_btn_raw_xpath ='//*[@id="app"]/div[1]/div[1]/div/div/div[1]/div/div/div/div/div[2]/button' 


    webEl_search_btn = driver.find_element(By.XPATH,search_btn_raw_xpath)
    
    print(webEl_search_btn.text)
    webEl_search_btn.click()

In [ ]:
def crawling_result(df_result:pd.DataFrame):
    sleep(1)
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'search-result__list'))
    )
    webEl_search_result_list = driver.find_element(By.CLASS_NAME, 'search-result__list')
    webEl_search_result_items = webEl_search_result_list.find_elements(By.TAG_NAME, 'li')

    '''
    <button>
    <ul class='search-info-list'>
    <ul class='search-info-list'>
    <ul class='btn__list'>
    '''

    for search_result_item in webEl_search_result_items:
        try:
            result_item = []
            title = search_result_item.find_element(By.TAG_NAME, 'button').text
            result_item.append(title)
            print(title)
            info_list = search_result_item.find_elements(By.CLASS_NAME, 'search-info-list')
            for info_items in info_list:
                info_item = info_items.find_elements(By.TAG_NAME, 'li')
                for li in info_item:
                    info_title = li.find_element(By.CLASS_NAME, 'list-tit')
                    info_info = li.find_element(By.CLASS_NAME, 'list-info')
                    print(info_title.text, info_info.text)

                    result_item.append(info_info.text)

            df_result.loc[len(df_result)] = result_item
        except Exception:
            pass

    return df_result

In [ ]:
def set_current_index():
    index_div_raw_xpath = '//*[@id="app"]/div[1]/div[1]/div/div/div[2]/div[2]/div[2]/div/div'
    index_div_xpath = driver.find_element(By.XPATH,index_div_raw_xpath)
    webEl_index_btn = index_div_xpath.find_element(By.TAG_NAME, 'ul').find_elements(By.TAG_NAME,'li')
    for index_btn in webEl_index_btn:
        try:
            webEl_current_index =index_btn.find_element(By.CLASS_NAME, 'on') 
            current_index = int(webEl_current_index.text) % 10
            print('on: ', current_index)
            return current_index

        except Exception:
            print('off: ', index_btn.text)

In [ ]:
def is_end_page(webEl):
    if (webEl.get_attribute('disabled')):
        return True
    else:
        return False

In [ ]:
# //*[@id="app"]/div[1]/div[1]/div/div/div[2]/div[2]/div[2]/div/div/ul/li[2]/button # sec btn
# //*[@id="app"]/div[1]/div[1]/div/div/div[2]/div[2]/div[2]/div/div/ul/li[3]/button # 3th btn

def click_next_index(current_index):
    next_index = str(current_index+1)
    print("current index: ",current_index)
    print("next index: ",next_index)
    next_index_btn_xpath = driver.find_element(By.XPATH,'//*[@id="app"]/div[1]/div[1]/div/div/div[2]/div[2]/div[2]/div/div/button[4]')


    if is_end_page(next_index_btn_xpath):
        print('마지막 페이지')
        return False
    elif current_index == 0:
        print('마지막 번호')

    else:
        next_index_btn_raw_xpath = '//*[@id="app"]/div[1]/div[1]/div/div/div[2]/div[2]/div[2]/div/div/ul/li[{}]/button'.format(next_index)
        next_index_btn_xpath = driver.find_element(By.XPATH,next_index_btn_raw_xpath)
        
    next_index_btn_xpath.send_keys(Keys.ENTER)
    return True

In [ ]:
target_region = '서울'
target_option = ['정상','대기업','중견기업','유가증권시장']
columns_name = ['회사명','대표자명','기업상태','기업유형/형태','사업자번호','법인번호','산업분류','주소']
df_result = pd.DataFrame(columns=columns_name)


In [ ]:
select_search_option(target_option,target_region)
click_search()
while True:
    df_result = crawling_result(df_result)
    current_index = set_current_index()
    if not click_next_index(current_index):
        break

In [ ]:
print(df_result['회사명'].count())
df_result.tail()

In [ ]:
base_url = './'
file_name = 'result'+'.xlsx'
region = target_region
df_result.to_excel(excel_writer=base_url+file_name,sheet_name=region)

In [ ]:
# 브라우저 탭 닫기
driver.close()
# 브라우저 종료하기 (탭 모두 종료)
driver.quit()